In [72]:
import subprocess
import os

netlist_template = """
.include ../model/32nm_HP.pm
.param SUPPLY=1

.PARAM Lmin=32n
.PARAM Wmin=32n
.PARAM Ldiff=64n

.PARAM N={n}


.TEMP 85

M1 drain gate source body pmos W={{N*Wmin}} L={{Lmin}} AS={{N*Wmin*Ldiff}} AD={{N*Wmin*Ldiff}} PS={{2*(Ldiff+N*Wmin)}} PD={{2*(Ldiff+N*Wmin)}}
Vd 	drain	0		0
Vg 	gate	0		1
Vs 	source	alim		0
Vb 	body	1		1
Vdd 	alim	0		0

.control
let voltage=-0.01
let Vddbasic=1.0

while voltage le Vddbasic
  let voltage = voltage + 0.01
  alter Vdd = voltage
  dc TEMP 85 86 10
  print abs(V(alim)) V(drain) V(gate) V(source) V(body) I(Vd) I(Vg) I(Vs) I(Vb)

end

.endc

.end
"""

n_values = [1, 2, 3, 4, 6, 8]

output_dir = "temp"

# make the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

outs = []
for n in n_values:
    netlist_content = netlist_template.format(n=n)
    
    filename = f"{output_dir}/g1ssd0-pmos_{n}.net"
    
    with open(filename, 'w') as file:
        file.write(netlist_content)
    
    p = subprocess.run(["ngspice", filename] , encoding='utf-8', 
stdout=subprocess.PIPE)
    windstr = p.stdout
    outs.append(windstr)

Note: No ".plot", ".print", or ".fourier" lines; no simulations run
Note: No ".plot", ".print", or ".fourier" lines; no simulations run
Note: No ".plot", ".print", or ".fourier" lines; no simulations run
Note: No ".plot", ".print", or ".fourier" lines; no simulations run
Note: No ".plot", ".print", or ".fourier" lines; no simulations run
Note: No ".plot", ".print", or ".fourier" lines; no simulations run


In [73]:
lines = windstr.split('\n')
# make a matrix out of every 10-18 lines
matrix = []
for o in outs:
    lines = o.split('\n')
    curr = []
    # make a matrix out of every 10-18 lines
    for i in range(10, len(lines), 13):
        curr.append(lines[i:i+9])
    matrix.append(curr)

In [74]:
# make directory 
dir_name = "results-pmos"
os.makedirs(dir_name, exist_ok=True)

filename = f"{dir_name}/g1ssd0-pmos_width_"
for i, n in enumerate(n_values):
    with open(filename + str(n) + ".csv", 'w') as file:
        file.write("Valim,Vd,Vg,Vs,Vb,Id,Ig,Is,Ib\n") 
        for j in range(len(matrix[i])):
            for k in range(9):
                value = matrix[i][j][k].split(' ')[-1]
                file.write(value + ",")
                
            file.write("\n")